In [ ]:
!pip install langchain langchain-community langchain-core langchain-text-splitters chromadb
!pip install huggingface-hub protobuf==3.20.3
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.36.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
import os
import pathlib
import requests
import json
from typing import Dict, List
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain_community.chat_models import ChatLlamaCpp
from huggingface_hub import hf_hub_download


In [ ]:
def load_vectorstore(persist_dir: str, pdf_path: str, pdf_url: str, corpus: List[Document] = None):
    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    if os.path.exists(persist_dir):
        print(f"Loading existing vectorstore from: {persist_dir}")
        return Chroma(persist_directory=persist_dir, embedding_function=embedding_model)

    print(f"Creating new vectorstore at: {persist_dir}")
    os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

    if corpus:
        documents = corpus
    else:
        if not os.path.exists(pdf_path):
            print(f"Downloading PDF from {pdf_url}")
            response = requests.get(pdf_url)
            with open(pdf_path, "wb") as f:
                f.write(response.content)

        loader = PyPDFLoader(pdf_path)
        documents = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(documents)

    vectorstore = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory=persist_dir)
    vectorstore.persist()

    print(f"Vectorstore created and saved at: {persist_dir}")
    return vectorstore


In [ ]:
def update_sources_yaml(yaml_file: str, new_entry: dict):
    sources = []
    if os.path.exists(yaml_file):
        with open(yaml_file, "r", encoding="utf-8") as f:
            sources = yaml.safe_load(f) or []

    updated = False
    for i, entry in enumerate(sources):
        if entry.get("name") == new_entry.get("name"):
            sources[i] = new_entry
            updated = True
            break

    if not updated:
        sources.append(new_entry)

    with open(yaml_file, "w", encoding="utf-8") as f:
        yaml.safe_dump(sources, f, allow_unicode=True)

    print(f"Updated sources.yaml with: {new_entry}")


In [ ]:
# 1. Tạo hoặc load vectorstore
vectorstore = load_vectorstore(
    persist_dir="db/ipcc_store",
    pdf_path="data/ipcc_ar6.pdf",
    pdf_url="https://www.ipcc.ch/report/ar6/syr/downloads/report/IPCC_AR6_SYR_LongerReport.pdf"
)

# 2. Cập nhật sources.yaml
update_sources_yaml(
    yaml_file="data/sources.yaml",
    new_entry={"name": "IPCC_AR6", "path": "data/ipcc_ar6.pdf", "url": "https://www.ipcc.ch/report/ar6/syr/downloads/report/IPCC_AR6_SYR_LongerReport.pdf"}
)


/tmp/ipython-input-3200302988.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating new vectorstore at: db/ipcc_store


Vectorstore created and saved at: db/ipcc_store


/tmp/ipython-input-3200302988.py:27: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


NameError: name 'yaml' is not defined